In [200]:
import pickle
import pandas as pd

import sys
sys.path.append("..")
from src.support_prep import *

In [201]:
with open('../models/target.pkl', 'rb') as file:
    target = pickle.load(file)
with open('../models/onehot.pkl', 'rb') as file:
    onehot = pickle.load(file)
# Scaling
with open('../models/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
# Model
with open('../models/model_resampled.pkl', 'rb') as file:
    model = pickle.load(file)

In [202]:
cat_diff  = ['BusinessTravel',
            'Department',
            'EducationField',
            'JobRole',
            'MaritalStatus',
            'EnvironmentSatisfaction',
            'JobSatisfaction',
            'WorkLifeBalance',
            'JobInvolvement']

cat_no_diff = ['Education', 'Gender', 'JobLevel', 'StockOptionLevel','PerformanceRating']

In [ ]:
test_series = pd.read_pickle("../datos/clean.pkl").loc[[0]].drop(columns = ["Attrition", "EmployeeID"])
test_series

for i in test_series:
    try:
        print(int(i))
    except:

        print(i)

ValueError: too many values to unpack (expected 2)

In [204]:
import requests

requests.post(url = "http://127.0.0.1:5000/predict", json = test_series.loc[0].to_json())

<Response [500]>

In [205]:
res_target = target.transform(test_series)
res_target

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51.0,0.149569,0.150224,6.0,2.0,0.166667,Female,1.0,0.145038,0.124814,...,1.0,6.0,1.0,0.0,0.0,0.137778,0.114119,0.167812,0.153226,3.0


In [213]:
onehot.transform(res_target[cat_no_diff]).toarray()

array([[0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0.]])

In [208]:
res_onehot =pd.DataFrame(onehot.transform(res_target[cat_no_diff]).toarray(), columns = onehot.get_feature_names_out())

In [209]:
res_encoded = pd.concat([res_target.drop(columns = cat_no_diff), res_onehot], axis = 1)

In [210]:
res_scaled = pd.DataFrame(scaler.transform(res_encoded.drop(columns = res_onehot.columns)), columns = scaler.get_feature_names_out())
res_scaled = pd.concat([res_scaled, res_onehot], axis = 1)

In [211]:
model.predict(res_scaled)

array([0])